In [ ]:
ls ../scripts/toy_studies_50ns_inclusive/fit_result_pfCombinedInclusiveSecondaryVertexV2BJetTags_0.97_100.0_0.r

In [ ]:
from ROOT import TFile
import numpy as np
import corner


dir_path = "../scripts/toy_studies_50ns_pt_dependent//"
tag_dict = { "CSVv2L": ["pfCombinedInclusiveSecondaryVertexV2BJetTags", 0.604 ],
             "CSVv2M": ["pfCombinedInclusiveSecondaryVertexV2BJetTags", 0.890 ],
             "CSVv2T": ["pfCombinedInclusiveSecondaryVertexV2BJetTags", 0.970 ]}

In [ ]:
n_toys = 100

s_name = "CSVv2L"
tagger = tag_dict[s_name][0]
wp = tag_dict[s_name][1]
lumi = 300.0

bins = [30.0, 50.0, 70.0, 100.0, 140.0]


pars = np.zeros((n_toys, 2+(len(bins)-1) ))
errs = np.zeros((n_toys, 2+(len(bins)-1) ))
truths = np.zeros((2+(len(bins)-1) ))

for n in range(n_toys):          
    root_file = TFile("{}fit_result_{}_{}_{}_{}.root".format(dir_path, tagger, wp, lumi, n))
    
    fit_result = root_file.Get("fitresult_sim_kin_pdf_hmaster")
    i_pars = fit_result.floatParsInit()
    f_pars = fit_result.floatParsFinal()
    for p in range(f_pars.getSize()):
        pars[n][p] = f_pars[p].getVal()
        errs[n][p] = f_pars[p].getError()
    if n is 0:
        for p in range(i_pars.getSize()):
            truths[p] = i_pars[p].getVal()
    root_file.Close()
        



In [ ]:
%matplotlib inline

import matplotlib

matplotlib.rcParams.update({'font.size': 12,
                            'figure.figsize' : (20.0, 16.0) })
labels = [r"efficiency $ \epsilon^{0}_b$".format(i)for i in range(pars.shape[1]-2)] + [r"bkg norm $\kappa$", r"$\sigma_{t\bar{t}}$ /pb"]
print labels
matplotlib.rc("savefig", dpi=250)
figure = corner.corner(pars,
                       labels=labels,
                       truths = truths
                       )

figure.savefig("toy_corner_pt_{}_{:.0f}.pdf".format(s_name,lumi))



In [ ]:
n_toys = 50

taggers = {"CSVv2L" : {},
           "CSVv2M" : {},
           "CSVv2T" : {}}
           
for k,v in taggers.items():
    tagger = tag_dict[k][0]
    wp = tag_dict[k][1]
    lumi = 300.0
    
    bins = [30.0, 50.0, 70.0, 100.0, 140.0]
    
    
    pars = np.zeros((n_toys, 2+(len(bins)-1) ))
    errs = np.zeros((n_toys, 2+(len(bins)-1) ))
    truths = np.zeros((2+(len(bins)-1) ))
    
    for n in range(n_toys):
        if n is 0:
            for p in range(i_pars.getSize()):
                truths[p] = i_pars[p].getVal()
                
        root_file = TFile("{}fit_result_{}_{}_{}_{}.root".format(dir_path, tagger, wp, lumi, n))
        
        fit_result = root_file.Get("fitresult_sim_kin_pdf_hmaster")
        i_pars = fit_result.floatParsInit()
        f_pars = fit_result.floatParsFinal()
        for p in range(f_pars.getSize()):
            pars[n][p] = f_pars[p].getVal()
            errs[n][p] = f_pars[p].getError()
        root_file.Close()
    taggers[k]["pars"] = pars
    taggers[k]["errs"] = errs
    taggers[k]["truths"] = truths

In [ ]:
n_toys = 120

lumis = {"100.0" : {},
         "300.0" : {},
         "1000.0" : {}}
#lumis = {"100.0" : {}}

s_name = "CSVv2L"

for k,v in lumis.items():
    lumi = k
    tagger = tag_dict[s_name][0]
    wp = tag_dict[s_name][1]
    
    bins = [30.0, 50.0, 70.0, 100.0, 140.0]
    
    
    pars = np.zeros((n_toys, 2+(len(bins)-1) ))
    errs = np.zeros((n_toys, 2+(len(bins)-1) ))
    truths = np.zeros((2+(len(bins)-1) ))
    
    for n in range(n_toys):          
        root_file = TFile("{}fit_result_{}_{}_{}_{}.root".format(dir_path, tagger, wp, lumi, n))
        
        fit_result = root_file.Get("fitresult_sim_kin_pdf_hmaster")
        i_pars = fit_result.floatParsInit()
        f_pars = fit_result.floatParsFinal()
        for p in range(f_pars.getSize()):
            pars[n][p] = f_pars[p].getVal()
            errs[n][p] = f_pars[p].getError()
        if n is 0:
            for p in range(i_pars.getSize()):
                truths[p] = i_pars[p].getVal()
        root_file.Close()
    lumis[k]["pars"] = pars
    lumis[k]["errs"] = errs
    lumis[k]["truths"] = truths

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib

matplotlib.rcParams.update({'font.size': 11})
par_names = [r"efficiency $ \epsilon^{0}_b$".format(i)for i in range(pars.shape[1]-2)] + [r"bkg norm $\kappa$", r"$\sigma_{t\bar{t}}$ /pb"]
pull_names = [r"pull ($ \epsilon^{0}_b$)".format(i)for i in range(pars.shape[1]-2)] + [r"pull ($\kappa$)", r"pull ($\sigma_{t\bar{t}})$"]

f, axarr = plt.subplots(2, pars.shape[1], figsize=(32,12), sharey=True)
n_bins = 10
y_max = 40
for par in range(pars.shape[1]):
    for k,v in lumis.items():
        y_par, edg_par = np.histogram(v["pars"][:,par], bins = n_bins)
        axarr[0,par].errorbar(0.5*(edg_par[1:]-edg_par[1:])+edg_par[1:], y_par,
                              xerr =0.5*(edg_par[1:]-edg_par[1:]),
                              yerr = np.sqrt(y_par),
                              fmt = "o", label = "L = {:4.0f}".format(float(k))+" ${pb}^{-1}$" )
        axarr[0,par].set_ylim(0,y_max)
        y_pull, edg_pull = np.histogram((v["pars"][:,par] - v["truths"][par])/v["errs"][:,par], bins = n_bins)
        axarr[1,par].errorbar(0.5*(edg_pull[1:]-edg_pull[1:])+edg_pull[1:], y_pull,
                              xerr =0.5*(edg_pull[1:]-edg_pull[1:]),
                              yerr = np.sqrt(y_par), fmt = "o",
                             label = "L = {:4.0f}".format(float(k))+" ${pb}^{-1}$")
        axarr[1,par].set_ylim(0,y_max)
    axarr[0,par].set_xlabel(par_names[par])
    axarr[0,par].set_ylabel("# toys")
    axarr[0,par].legend(numpoints=1, frameon=False, prop={'size':13})
    axarr[1,par].set_xlabel(pull_names[par])
    axarr[1,par].set_ylabel("# toys")
    axarr[1,par].legend(numpoints=1, frameon=False, prop={'size':13})
      
#f.savefig("toy_pars_pull_pt_13tev_50ns.pdf")



In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib

matplotlib.rcParams.update({'font.size': 11})
par_names = [r"efficiency $ \epsilon^{0}_b$".format(i)for i in range(pars.shape[1]-2)] + [r"bkg norm $\kappa$", r"$\sigma_{t\bar{t}}$ /pb"]
pull_names = [r"pull ($ \epsilon^{0}_b$)".format(i)for i in range(pars.shape[1]-2)] + [r"pull ($\kappa$)", r"pull ($\sigma_{t\bar{t}})$"]


n_bins = 10
y_max = 40

iterate =  lumis.items()[1:2]
print iterate[0][0], " & par", 
for par in range(pars.shape[1]):
    for k,v in iterate:
        if par != 5:
            print "& ${:.3f}\pm{:.3f}$ ".format(np.mean(v["pars"][:,par]), np.std(v["pars"][:,par])/np.sqrt(n_toys)),
        else:    
            print "& ${:.0f}\pm{:.0f}$ ".format(np.mean(v["pars"][:,par]), np.std(v["pars"][:,par])/np.sqrt(n_toys)),

        
print "\\\\"

print iterate[0][0], " & err", 
for par in range(pars.shape[1]):
    for k,v in iterate:
        if par != 5:
            print "& ${:.5f}\pm{:.5f}$ ".format(np.mean(v["errs"][:,par]), np.std(v["errs"][:,par])/np.sqrt(n_toys)),
        else:
            print "& ${:.2f}\pm{:.2f}$ ".format(np.mean(v["errs"][:,par]), np.std(v["errs"][:,par])/np.sqrt(n_toys)),
            

print "\\\\"
print iterate[0][0], " & pull", 
for par in range(pars.shape[1]):
    for k,v in iterate:
        pull_mean = np.mean((v["pars"][:,par] - v["truths"][par])/v["errs"][:,par]) 
        pull_std = np.std((v["pars"][:,par] - v["truths"][par])/v["errs"][:,par]) 
        pull_mean_err =  pull_std/np.sqrt(n_toys)
        pull_std_err =  pull_std*np.sqrt(2./(n_toys-1))/2.
        print "& ${:+.2f}\pm{:.2f}$ ".format(pull_mean, pull_mean_err),
print "\\\\"


print "\\\\"
print iterate[0][0], " & pull", 
for par in range(pars.shape[1]):
    for k,v in iterate:
        pull_mean = np.mean((v["pars"][:,par] - v["truths"][par])/v["errs"][:,par]) 
        pull_std = np.std((v["pars"][:,par] - v["truths"][par])/v["errs"][:,par]) 
        pull_mean_err =  pull_std/np.sqrt(n_toys)
        pull_std_err =  pull_std*np.sqrt(2./(n_toys-1))/2.
        print "& ${:+.2f}\pm{:.2f}$ ".format(pull_std, pull_std_err),
print "\\\\"

for par in range(pars.shape[1]):
    for k,v in iterate:

        print "& ${:+.3f}$ ".format(v["truths"][par]),
print "\\\\"

In [ ]:
pars

In [ ]:
lumi = "1000.0"
lumis[lumi]["errs"]/lumis[lumi]["pars"]